In [1]:
import os

In [2]:
!pwd

/home/jovyan/notebooks/analysis


In [3]:
import sys
sys.path.append('/home/jovyan/')

In [4]:
from experiments.reporting.report_repository import ReportRepository
from experiments.slack_client import SlackNotifier
from neat.neat_logger import get_neat_logger
from neat.population_engine import EvolutionEngine
from neat.reporting.reports_pyneat import EvolutionReport
from neat.utils import timeit
import math
import random
from itertools import count
import numpy as np

from experiments.logger import logger
from experiments.slack_client import Notifier
from experiments.object_repository.object_repository import ObjectRepository
from experiments.reporting.report_repository import ReportPathFactory
from neat.configuration import get_configuration
from neat.evaluation.evaluation_engine import EvaluationStochasticEngine
from neat.evolution_operators.crossover import Crossover
from neat.evolution_operators.mutation import Mutation
from neat.genome import Genome
from neat.reporting.reports_pyneat import EvolutionReport
from neat.species import SpeciationEngine
from neat.stagnation import Stagnation
from neat.population_engine import PopulationEngine
from neat.utils import timeit
import neat
import tests
from config_files.configuration_utils import create_configuration
from neat.configuration import get_configuration, set_configuration
from neat.evaluation.evaluation_engine import get_dataset
from neat.evaluation.evaluate_simple import evaluate_genome, calculate_prediction_distribution
from neat.genome import Genome
from neat.evolution_operators.backprop_mutation import BackPropMutation
import os
from neat.loss.vi_loss import get_loss
import torch
import pandas as pd

from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, roc_curve

In [5]:
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
from neat.analysis.experiment_data import ExperimentDataNE, ExperimentDataNAS
from neat.analysis.uncertainty.predictive_distribution import PredictionDistributionEstimator
from neat.analysis.plotting.plot_losses import plot_loss_distribution
from neat.analysis.plotting.plot_classification_metrics import plot_metrics_by_quantile_several_executions
from neat.analysis.experiment_predictive_distributions import ExecutionsPredictionDistributions
from deep_learning.nas import ALGORITHM_VERSION as ALGORITHM_VERSION_NAS

In [7]:
LOGS_PATH = f'{os.getcwd()}/'
logger = get_neat_logger(path=LOGS_PATH)

In [8]:
PROJECT = 'neuro-evolution'
ALGORITHM_VERSION = 'bayes-neat'
project_nas='nas'
algorithm_version_nas ='nas'

config_versions = {'NE': {'project': 'neuro-evolution',
                         'algorithm_version': 'bayes-neat'},
                  'NAS': {'project': 'nas',
                         'algorithm_version': 'nas'}
                  }

# correlation_ids = [f'neat_ft_21_{DATASET}', f'bayesian_neat_ft_21_{DATASET}'] # final

correlation_ids_map = {'iris': {
#                                 'NE': [f'neat_ft_exp_iris', f'bayesian_neat_ft_exp_iris'],
                                'NE': [f'neat_ft_21_iris', f'bayesian_neat_ft_21_iris'],
                                'NAS': ['standard_nas_final_with_reps', 'bayesian_nas_final']},
                       'titanic': {
                                'NE': [f'neat_ft_22_final_titanic', f'bayesian_neat_ft_22_final_titanic'],
                                'NAS': ['standard_nas_final_with_reps', 'bayesian_nas_final_new_layer']},
                       'spambase': {
                                'NE': [f'neat_ft_22_final_spambase', f'bayesian_neat_ft_22_final_spambase'],
                                'NAS': ['standard_nas_final_with_reps', 'bayesian_nas_final_new_layer']},
                       }
                       
                      

# correlation_ids = [f'neat_ft_18_{DATASET}'] # All executions use random dataset-seed

In [10]:
version = 'NE'
report_repository = ReportRepository.create(project=config_versions[version]['project'], logs_path=LOGS_PATH)
list(report_repository.get_correlation_ids(algorithm_version=config_versions[version]['algorithm_version'], dataset='mnist_downsampled'))

['neat_ft_final_v1_label_noise_mnist_downsampled',
 'bayesian_neat_ft_final_v1_mnist_downsampled',
 'bayesian_neat_ft_18_mnist_downsampled',
 'bayesian_neat_ft_22_debug_mnist_downsampled',
 'neat_ft_1_mnist_downsampled',
 'neat_ft_18_mnist_downsampled',
 'bayesian_neat_ft_20_mnist_downsampled',
 'tests_new_network_mnist_downsampled',
 'bayesian_neat_ft_21_debug_mnist_downsampled',
 'bayesian_neat_10_mnist_downsampled',
 'neat_10_mnist_downsampled',
 'neat_ft_17_mnist_downsampled',
 'test',
 'neat_1',
 'bayesian_neat_ft_22_final_1_mnist_downsampled',
 'bayesian_neat_ft_final_v1_5_classesmnist_downsampled',
 'neat_ft_22_final_1_mnist_downsampled',
 'test_6mnist_downsampled',
 'bayesian_neat_1',
 'neat_ft_final_v1_5_classesmnist_downsampled',
 'bayesian_neat_ft_22_lenovo_final_1_mnist_downsampled',
 'tests',
 'neat_ft_final_v1_mnist_downsampled',
 'bayesian_neat_ft_22_testing_mnist_downsampled',
 'bayesian_neat_ft_17_mnist_downsampled',
 'solve_binary',
 'bayesian_neat_ft_22__mnist_downsa

In [10]:
report_repository = ReportRepository.create(project=config_versions[version]['project'], logs_path=LOGS_PATH)
len(list(report_repository.object_repository.tree('reports')))

3353

In [36]:
executions = list(report_repository.get_executions(algorithm_version=config_versions[version]['algorithm_version'], dataset='breast_cancer', correlation_id='bayesian_neat_ft_20_breast_cancer'))
display(len(executions))

55

# NAS

In [30]:
version = 'NAS'

In [31]:
report_repository = ReportRepository.create(project=config_versions[version]['project'], logs_path=LOGS_PATH)
list(report_repository.get_correlation_ids(algorithm_version=config_versions[version]['algorithm_version'], dataset='breast_cancer'))

['standard_nas_final', 'bayesian_nas_final']

In [24]:
report_repository = ReportRepository.create(project=config_versions[version]['project'], logs_path=LOGS_PATH)
executions = list(report_repository.get_executions(algorithm_version=config_versions[version]['algorithm_version'], dataset='titanic', correlation_id='bayesian_nas_final_new_layer'))
display(len(executions))
executions

28

['033c10aa-7a3e-4dcf-bb59-cbb020b63986',
 '1f1ece0e-8198-4ace-b27e-fc35529a8d3f',
 '20de3188-e777-4bcd-9a81-a332bceb18dc',
 '25b4ba8e-c2b5-4716-97a0-8b06c9f74e77',
 '3170ef22-ac25-4908-bbd5-77f3f6d0aaab',
 '31cb365c-6816-47ca-a7e0-e842e035b5a2',
 '34fbfdbf-7eaf-4139-a67e-91fa6df8b697',
 '3d4421f4-8f0f-4a6b-ba9a-ab336565efe4',
 '5c6cb8d2-6c0b-4700-9789-7ba671d102d2',
 '5dad3265-d269-4bba-9fc6-e9fb4a558957',
 '6926843c-65f0-44a9-b31f-e02d6ca827ab',
 '6a9c1631-7246-4708-bdcb-988562cb8ae9',
 '83e727f7-f206-468c-bfc2-aeba0ed63ba1',
 '9dc13139-f836-4f23-a6ce-89e5abca4c00',
 'a16bf166-f260-4eca-b6f5-4ed6fbb80c99',
 'a29b55f2-703c-4574-903e-0ed1bb2645e9',
 'aeeffd2b-6a05-46d4-b0ee-dc1511c7ede0',
 'bf657bf3-3170-4692-a642-9b41d85e5557',
 'bfa07892-6a5c-4f30-94f2-ab8774e96ef6',
 'c8338403-cd92-4c11-871d-18707ecf5223',
 'ce2e174c-9969-4ce1-aeb7-12f32dd49a56',
 'd49625cb-cf68-4504-92bc-fb44f6bb04fb',
 'd5bc3083-e951-4918-b791-e2c0cbb480ec',
 'd73611c3-58c3-45a5-acb0-0855633d3bb9',
 'dc8a948a-4172-

In [39]:
executions = list(report_repository.get_executions(algorithm_version=config_versions[version]['algorithm_version'], dataset='titanic', correlation_id='bayesian_neat_ft_22_final_titanic'))
display(len(executions))
executions

10

['05a0bb77-b50a-42fa-8374-adc75b911beb',
 '21e471ec-5aef-4a7f-9eb0-c1ed854e3963',
 '292b93d0-b82a-48c6-86c2-d1617e921439',
 '37ab6688-dde4-4960-979f-58abc2864708',
 '78e825c4-7023-4e06-a20b-23d4d29004cf',
 '8ee02077-92aa-419d-81da-de6f285d33d8',
 '8ff16818-b7dd-42d4-b34e-c70adfcca961',
 '940f01cb-36c8-4fe7-9b04-dc756fbde98e',
 'e3edbf25-0960-405a-a047-418b00d93ce2',
 'ebec36cb-cbd8-4043-8c57-0298cb312406']